In [1]:
import os
os.chdir('D:\Birkbeck\MSc Data Science\Project\Data sets\Selected datasets')

import pandas as pd
import numpy as np
import glob

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Creating Dataset with 'Static' values

### Size of Area in Hectares & Size of Green Belt

In [2]:
area = pd.read_csv('live table green belt and total area.csv', header = 3)
#area.head(10)

In [3]:
area = area.drop(columns = ['Unnamed: 5',
                            'Unnamed: 6', 
                            'Unnamed: 7',
                            'Unnamed: 8',
                            'Percentage of total land area designated as Green Belt'])
area = area[area['ONS code'].str.contains('E0') == True]

area = area.rename(columns = {'ONS code': 'la_code',
                             'Local authority': 'la_name',
                             'Area of land designated as Green Belt': 'g_belt_area',
                             'Total area as at 31 December 2020 2': 'total_area'})
area['g_belt_area'] = area['g_belt_area'].replace({'-':0})

area['g_belt_area'] = area['g_belt_area'].replace({',': ''}, regex=True).astype('int64')
area['total_area'] = area['total_area'].replace({',': ''}, regex=True).astype('int64')

# area.info()
# area.head(10)

In [6]:
# need to add years to be the same for all
for year in range(2000,2022):
    area[year] = year

area.head(10)

,la_code,la_name,g_belt_area,total_area,year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,E07000223,Adur,0,4210,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,E07000026,Allerdale,0,125830,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
2,E07000032,Amber Valley,8650,26540,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
3,E07000224,Arun,0,22100,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
4,E07000170,Ashfield,4520,10960,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
5,E07000105,Ashford,0,58060,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
6,E07000200,Babergh,0,59510,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
7,E09000002,Barking and Dagenham,530,3610,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
8,E09000003,Barnet,2380,8680,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
9,E08000016,Barnsley,22410,32910,2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021


In [7]:
area = pd.melt(area, id_vars = ['la_code', 'la_name', 'g_belt_area','total_area'], value_name = 'year')
area = area.drop(columns = 'variable')
area['la_name'] = area['la_name'].str.strip()
area['la_code'] = area['la_code'].str.strip()
area.head()

C:\Users\Michelle Bark\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  exec(code_obj, self.user_global_ns, self.user_ns)


,la_code,la_name,g_belt_area,total_area,year
0,E07000223,Adur,0,4210,2000
1,E07000026,Allerdale,0,125830,2000
2,E07000032,Amber Valley,8650,26540,2000
3,E07000224,Arun,0,22100,2000
4,E07000170,Ashfield,4520,10960,2000


In [6]:
move_year = area.pop('year')
area.insert(2,'year',move_year)
#area.info()

In [11]:
la_file = area

### Adding RPI by year

In [8]:
rpi = pd.read_csv('annual all items rpi.csv', header = 7)
#rpi.head()

In [9]:
#only need from 2000 to 2021
rpi = rpi.drop(rpi.index[35:])
rpi = rpi.drop(rpi.index[:13])
rpi = rpi.rename(columns ={'Important notes': 'year','Unnamed: 1': 'rpi'})

In [12]:
#Create list of codes and names to be able to add
code_lst = la_file['la_code'].unique().tolist()
name_lst = la_file['la_name'].unique().tolist()
#print(code_lst)
#print(name_lst)

In [ ]:
count = 0
for code in code_lst:
    ref = name_lst[count]
    rpi[ref] = code
    count+=1
rpi

In [14]:
rpi = pd.melt(rpi, id_vars = ['year', 'rpi'], var_name = 'la_name', value_name = 'la_code')
rpi['year'] = rpi['year'].astype('int64')
rpi['rpi'] = rpi['rpi'].astype('float')
rpi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6908 entries, 0 to 6907
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     6908 non-null   int64  
 1   rpi      6908 non-null   float64
 2   la_name  6908 non-null   object 
 3   la_code  6908 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 216.0+ KB


In [13]:
rpi.head()

,year,rpi,la_name,la_code
0,2000,170.3,Adur,E07000223
1,2001,173.3,Adur,E07000223
2,2002,176.2,Adur,E07000223
3,2003,181.3,Adur,E07000223
4,2004,186.7,Adur,E07000223


In [14]:
la_file = la_file.merge(rpi)
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6908 entries, 0 to 6907
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   la_code      6908 non-null   object 
 1   la_name      6908 non-null   object 
 2   year         6908 non-null   int64  
 3   g_belt_area  6908 non-null   int64  
 4   total_area   6908 non-null   int64  
 5   rpi          6908 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 377.8+ KB


### Adding BoE Base rate/Interest Rates

In [15]:
#Uses file created in england dataset with all years min/max
int_rate = pd.read_csv('pd_int_rate.csv')
int_rate = int_rate.drop(columns = 'Unnamed: 0')
#int_rate

In [16]:
# now add area names and codes for each to add to static info
count = 0
for code in code_lst:
    ref = name_lst[count]
    int_rate[ref] = code
    count+=1
    
int_rate = pd.melt(int_rate, id_vars = ['year', 'max', 'min'], var_name = 'la_name', value_name = 'la_code')
int_rate = int_rate.rename(columns = {'max': 'int_rate_max',
                                     'min': 'int_rate_min'})
#int_rate.info()

In [17]:
la_file = la_file.merge(int_rate)
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6908 entries, 0 to 6907
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   la_code       6908 non-null   object 
 1   la_name       6908 non-null   object 
 2   year          6908 non-null   int64  
 3   g_belt_area   6908 non-null   int64  
 4   total_area    6908 non-null   int64  
 5   rpi           6908 non-null   float64
 6   int_rate_max  6908 non-null   float64
 7   int_rate_min  6908 non-null   float64
dtypes: float64(3), int64(3), object(2)
memory usage: 485.7+ KB


### Adding: Median Income

In [18]:
med_income = pd.read_csv('5b Median Income by LA.csv', header = 1)
med_income.head()

,Region code,Region name,Code,Name,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,E12000001,North East,E06000001,Hartlepool,"15,747","16,149","16,121","15,743","17,264","18,251","17,153","17,258","20,850","20,478","19,913","21,047","21,298","23,762","23,430","24,741","27,220","25,143","26,445","26,112","25,638","25,729","26,010","28,093","28,378"
1,E12000001,North East,E06000002,Middlesbrough,"15,914","15,319","16,073","16,513","17,273","19,252","17,999","18,635","18,706","20,189","20,495","21,301","22,593","21,628","21,932","24,669","24,133","23,971","24,751","25,722","25,626","25,170","26,203","27,005","25,968"
2,E12000001,North East,E06000003,Redcar and Cleveland,"19,206","18,747","21,274","22,135","21,763","20,806","20,554","23,903","25,049","22,897","24,176","25,340","25,415","24,156","25,412","23,684","25,295","26,323","26,792","24,496","25,456","26,170","25,846","26,220","24,975"
3,E12000001,North East,E06000004,Stockton-on-Tees,"16,838","17,062","17,199","18,147","19,186","18,392","18,656","20,660","21,390","21,684","22,766","22,859","24,156","23,563","23,829","23,408","24,141","25,096","25,345","26,884","25,258","26,540","29,049","30,531","28,609"
4,E12000001,North East,E06000005,Darlington,"15,424","14,453","15,915","16,043","17,220","18,568","18,015","18,264","18,504","19,913","22,633","22,556","22,630","22,259","23,363","24,149","21,288","25,502","26,570","27,009","25,712","26,043","27,888","28,045","28,323"


In [19]:
med_income = med_income.drop(columns = ['Region code', 'Region name'])
med_income = med_income.rename(columns = {'Code': 'la_code', 'Name':'la_name'})
med_income = pd.melt(med_income, id_vars= ['la_code','la_name'], var_name = "year",value_name = 'median_income')
med_income = med_income.replace(":", np.nan)
med_income.dropna(inplace =True)
med_income['median_income']  = med_income['median_income'].replace({',': ''}, regex=True).astype('int64')
med_income['la_name'] = med_income['la_name'].str.strip()
med_income['la_code'] = med_income['la_code'].str.strip()
med_income['year'] = med_income['year'].astype('int64')
med_income.head()

,la_code,la_name,year,median_income
0,E06000001,Hartlepool,1997,15747
1,E06000002,Middlesbrough,1997,15914
2,E06000003,Redcar and Cleveland,1997,19206
3,E06000004,Stockton-on-Tees,1997,16838
4,E06000005,Darlington,1997,15424


In [20]:
la_file = la_file.merge(med_income)
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6657 entries, 0 to 6656
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   la_code        6657 non-null   object 
 1   la_name        6657 non-null   object 
 2   year           6657 non-null   int64  
 3   g_belt_area    6657 non-null   int64  
 4   total_area     6657 non-null   int64  
 5   rpi            6657 non-null   float64
 6   int_rate_max   6657 non-null   float64
 7   int_rate_min   6657 non-null   float64
 8   median_income  6657 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 520.1+ KB


### Functions for processing HPSSA files

In [21]:
#removing unwanted columns and renaming september data to only the year.
def rename_cols(x):
    if 'Year ending Dec' in x or 'Year ending Mar' in x or 'Year ending Jun' in x:
        return 'Remove'
    if 'Year ending Sep' in x:
        return x.split(' ')[3]
    return x

In [22]:
def file_processing(file,prop_type, measure):
    create_file = pd.read_csv(file, header = 6)
    create_file = create_file.drop(columns = ['Region/Country code', 'Region/Country name'])
    create_file = create_file.rename(columns = 
                                   {'Local authority code ': 'la_code', 
                                    'Local authority name':'la_name'})
    create_file = create_file.rename(columns = rename_cols).drop(columns = 'Remove').dropna()
    col_header = prop_type+'_'+measure
    create_file = pd.melt(create_file, id_vars= ['la_code', 'la_name'],
       var_name = "year",value_name = col_header )
    
    create_file = create_file.replace(":", np.nan)
    create_file.dropna(inplace =True)
    

    create_file[col_header]  = create_file[col_header].replace({',': ''}, regex=True).astype('int64')
    create_file['year'] = create_file['year'].astype('int64')
    create_file['la_name'] = create_file['la_name'].str.strip()
    create_file['la_code'] = create_file['la_code'].str.strip()
    return create_file

In [23]:
def merge_set(main_data_file,df_list):
    new_a = pd.merge(df_list[0],df_list[1])
    new_b = pd.merge(df_list[2],df_list[3])
    new_c = pd.merge(new_a, new_b)
    main_data_file = main_data_file.merge(new_c)
    return main_data_file

In [24]:
med_new_build_all = file_processing('hpssa 10 2a median new build prices.csv', 'all', 'new_sls_price')
med_new_build_det = file_processing('hpssa 10 2b median new build detached prices.csv', 'det', 'new_sls_price')
med_new_build_semi = file_processing('hpssa 10 2c median new build semi prices.csv','semi', 'new_sls_price')
med_new_build_terr = file_processing('hpssa 10 2d median new build terrace prices.csv','terr', 'new_sls_price')
med_new_build_flat = file_processing('hpssa 10 2e median new build flat prices.csv', 'flat','new_sls_price')

la_file = la_file.merge(med_new_build_all)
la_file = merge_set(la_file,[med_new_build_det, med_new_build_semi, med_new_build_terr, med_new_build_flat])

#la_file.head()                                

In [25]:
med_exist_all = file_processing('hpssa 11 2a median price exisiting.csv', 'all', 'exist_sls_price')
med_exist_det = file_processing('hpssa 11 2b median price exisiting detached.csv', 'det', 'exist_sls_price')
med_exist_semi = file_processing('hpssa 11 2c median price exisiting semi.csv','semi', 'exist_sls_price')
med_exist_terr = file_processing('hpssa 11 2d median price exisiting terrace.csv','terr', 'exist_sls_price')
med_exist_flat = file_processing('hpssa 11 2e median price exisiting flat.csv', 'flat','exist_sls_price')

la_file  = la_file .merge(med_exist_all)
la_file = merge_set(la_file ,[med_exist_det, med_exist_semi, med_exist_terr, med_exist_flat])
#la_file.head()    

In [26]:
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4902 entries, 0 to 4901
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   la_code               4902 non-null   object 
 1   la_name               4902 non-null   object 
 2   year                  4902 non-null   int64  
 3   g_belt_area           4902 non-null   int64  
 4   total_area            4902 non-null   int64  
 5   rpi                   4902 non-null   float64
 6   int_rate_max          4902 non-null   float64
 7   int_rate_min          4902 non-null   float64
 8   median_income         4902 non-null   int64  
 9   all_new_sls_price     4902 non-null   int64  
 10  det_new_sls_price     4902 non-null   int64  
 11  semi_new_sls_price    4902 non-null   int64  
 12  terr_new_sls_price    4902 non-null   int64  
 13  flat_new_sls_price    4902 non-null   int64  
 14  all_exist_sls_price   4902 non-null   int64  
 15  det_exist_sls_price  

In [27]:
med_all = file_processing('hpssa 9 2a median prices.csv', 'all', 'sls_price')
med_det = file_processing('hpssa 9 2b median detached prices.csv', 'det', 'sls_price')
med_semi = file_processing('hpssa 9 2c median semi prices.csv','semi', 'sls_price')
med_terr = file_processing('hpssa 9 2d median terrace prices.csv','terr', 'sls_price')
med_flat = file_processing('hpssa 9 2e median flat prices.csv', 'flat','sls_price')

la_file  = la_file .merge(med_all)
la_file  = merge_set(la_file ,[med_det, med_semi, med_terr, med_flat])
#la_file.info()  

In [28]:
num_new_build_all = file_processing('hpssa 7 2a num new build sls.csv', 'all', 'new_sls_num')
num_new_build_det = file_processing('hpssa 7 2b num new build detached sls.csv', 'det', 'new_sls_num')
num_new_build_semi = file_processing('hpssa 7 2c num new build semi sls.csv','semi', 'new_sls_num')
num_new_build_terr = file_processing('hpssa 7 2d num new build terrace sls.csv','terr', 'new_sls_num')
num_new_build_flat = file_processing('hpssa 7 2e num new build flat sls.csv', 'flat','new_sls_num')

la_file = la_file.merge(num_new_build_all)
la_file = merge_set(la_file,[num_new_build_det, num_new_build_semi, num_new_build_terr, num_new_build_flat])
#la_file.info()

In [29]:
num_exist_all = file_processing('hpssa 8 2a num existing sls.csv', 'all', 'exist_sls_num')
num_exist_det = file_processing('hpssa 8 2b num existing detached sls.csv', 'det', 'exist_sls_num')
num_exist_semi = file_processing('hpssa 8 2c num existing semi sls.csv','semi', 'exist_sls_num')
num_exist_terr = file_processing('hpssa 8 2d num existing terrace sls.csv','terr', 'exist_sls_num')
num_exist_flat = file_processing('hpssa 8 2e num existing flat sls.csv', 'flat','exist_sls_num')

la_file = la_file.merge(num_exist_all)
la_file = merge_set(la_file,[num_exist_det,num_exist_semi, num_exist_terr, num_exist_flat])
#la_file.info()  

In [30]:
num_all = file_processing('hpssa 6 2a num sales all.csv', 'all', 'sls_num')
num_det = file_processing('hpssa 6 2b num sales detached.csv', 'det', 'sls_num')
num_semi = file_processing('hpssa 6 2c num sales semi.csv','semi', 'sls_num')
num_terr = file_processing('hpssa 6 2d num sales terrace.csv','terr', 'sls_num')
num_flat = file_processing('hpssa 6 2e num sales flat.csv', 'flat','sls_num')

la_file = la_file.merge(num_all)
la_file = merge_set(la_file,[num_det, num_semi, num_terr, num_flat])
#la_file.info()

In [31]:
la_file.tail()

,la_code,la_name,year,g_belt_area,total_area,rpi,int_rate_max,int_rate_min,median_income,all_new_sls_price,det_new_sls_price,semi_new_sls_price,terr_new_sls_price,flat_new_sls_price,all_exist_sls_price,det_exist_sls_price,semi_exist_sls_price,terr_exist_sls_price,flat_exist_sls_price,all_sls_price,det_sls_price,semi_sls_price,terr_sls_price,flat_sls_price,all_new_sls_num,det_new_sls_num,semi_new_sls_num,terr_new_sls_num,flat_new_sls_num,all_exist_sls_num,det_exist_sls_num,semi_exist_sls_num,terr_exist_sls_num,flat_exist_sls_num,all_sls_num,det_sls_num,semi_sls_num,terr_sls_num,flat_sls_num
4897,E06000054,Wiltshire,2021,6910,325530,305.0,0.25,0.25,29069,319995,387000,272000,234995,284500,300000,425000,274000,235000,152000,300000,420000,273000,235000,155000,784,387,246,111,40,8992,3454,2323,2440,775,9776,3841,2569,2551,815
4898,E07000094,Winchester,2021,0,66100,305.0,0.25,0.25,33601,382500,472473,349000,341473,745000,475000,675000,412500,378265,234250,470500,665000,400000,370000,237250,155,58,41,40,16,2308,923,533,582,270,2463,981,574,622,286
4899,E06000041,Wokingham,2021,2900,17900,305.0,0.25,0.25,38014,440000,565530,430000,425000,279000,450000,595000,425000,350000,230000,445975,590000,425000,355000,236250,319,124,117,46,32,3129,1273,881,663,312,3448,1397,998,709,344
4900,E07000238,Wychavon,2021,8860,66350,305.0,0.25,0.25,28865,342000,364750,255000,252500,369500,297250,407500,252500,217200,130000,302500,400000,252500,219750,137000,207,116,54,22,15,2270,1075,609,458,128,2477,1191,663,480,143
4901,E06000014,York,2021,22410,27190,305.0,0.25,0.25,32490,350000,420000,332745,285000,324995,272000,395000,266000,245000,175000,275000,395000,268200,246000,180000,119,28,18,8,65,3681,831,1209,1167,474,3800,859,1227,1175,539


### Adding: Population

In [32]:
pop_la = pd.read_csv('pop by la.csv', header = 1)
pop_la.head()

'''2021 data has been added manually to the csv using the 2021 census data
Missing areas are filled with 0, these are NI and scotland which are not being used.'''

'2021 data has been added manually to the csv using the 2021 census data\nMissing areas are filled with 0, these are NI and scotland which are not being used.'

In [33]:
pop_la = pop_la.drop(columns = 'ITL1 Region')
pop_la = pop_la.rename(columns = {'LA code': 'la_code','LA name': 'la_name'})
pop_la = pd.melt(pop_la, id_vars = ['la_code', 'la_name'], var_name = 'year', value_name = 'population')
pop_la['population']  = pop_la['population'].replace({',': ''}, regex=True).astype('int64')
pop_la['year'] = pop_la['year'].astype('int64')
pop_la['la_code'] = pop_la['la_code'].str.strip()
pop_la['la_name'] = pop_la['la_name'].str.strip()
#pop_la.head()   

In [34]:
la_file = la_file.merge(pop_la)
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4902 entries, 0 to 4901
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   la_code               4902 non-null   object 
 1   la_name               4902 non-null   object 
 2   year                  4902 non-null   int64  
 3   g_belt_area           4902 non-null   int64  
 4   total_area            4902 non-null   int64  
 5   rpi                   4902 non-null   float64
 6   int_rate_max          4902 non-null   float64
 7   int_rate_min          4902 non-null   float64
 8   median_income         4902 non-null   int64  
 9   all_new_sls_price     4902 non-null   int64  
 10  det_new_sls_price     4902 non-null   int64  
 11  semi_new_sls_price    4902 non-null   int64  
 12  terr_new_sls_price    4902 non-null   int64  
 13  flat_new_sls_price    4902 non-null   int64  
 14  all_exist_sls_price   4902 non-null   int64  
 15  det_exist_sls_price  

Note: lost some rows - checked that it's not because of bad refs to la_code and la_name

In [35]:
la_file.tail()

,la_code,la_name,year,g_belt_area,total_area,rpi,int_rate_max,int_rate_min,median_income,all_new_sls_price,det_new_sls_price,semi_new_sls_price,terr_new_sls_price,flat_new_sls_price,all_exist_sls_price,det_exist_sls_price,semi_exist_sls_price,terr_exist_sls_price,flat_exist_sls_price,all_sls_price,det_sls_price,semi_sls_price,terr_sls_price,flat_sls_price,all_new_sls_num,det_new_sls_num,semi_new_sls_num,terr_new_sls_num,flat_new_sls_num,all_exist_sls_num,det_exist_sls_num,semi_exist_sls_num,terr_exist_sls_num,flat_exist_sls_num,all_sls_num,det_sls_num,semi_sls_num,terr_sls_num,flat_sls_num,population
4897,E06000054,Wiltshire,2021,6910,325530,305.0,0.25,0.25,29069,319995,387000,272000,234995,284500,300000,425000,274000,235000,152000,300000,420000,273000,235000,155000,784,387,246,111,40,8992,3454,2323,2440,775,9776,3841,2569,2551,815,510400
4898,E07000094,Winchester,2021,0,66100,305.0,0.25,0.25,33601,382500,472473,349000,341473,745000,475000,675000,412500,378265,234250,470500,665000,400000,370000,237250,155,58,41,40,16,2308,923,533,582,270,2463,981,574,622,286,127500
4899,E06000041,Wokingham,2021,2900,17900,305.0,0.25,0.25,38014,440000,565530,430000,425000,279000,450000,595000,425000,350000,230000,445975,590000,425000,355000,236250,319,124,117,46,32,3129,1273,881,663,312,3448,1397,998,709,344,177500
4900,E07000238,Wychavon,2021,8860,66350,305.0,0.25,0.25,28865,342000,364750,255000,252500,369500,297250,407500,252500,217200,130000,302500,400000,252500,219750,137000,207,116,54,22,15,2270,1075,609,458,128,2477,1191,663,480,143,132500
4901,E06000014,York,2021,22410,27190,305.0,0.25,0.25,32490,350000,420000,332745,285000,324995,272000,395000,266000,245000,175000,275000,395000,268200,246000,180000,119,28,18,8,65,3681,831,1209,1167,474,3800,859,1227,1175,539,202800


### Adding: Household Type

In [36]:
num_hh_type = pd.read_csv('Stage 2 projected households - Principal (type).csv')
num_hh_type.head()

,CODE,AREA NAME,AGE GROUP,HOUSEHOLD TYPE,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043
0,E92000001,England,16-19,One person households: Male,19566.011690,19810.083870,19856.808470,19814.291230,19524.052160,19533.359330,19673.367530,19455.333340,19232.919300,18997.929960,18223.858800,17720.087410,17328.954580,16912.998820,16625.527700,16197.129210,15606.341790,15112.578790,14700.926690,14524.160110,14519.340550,15000.654000,15507.038040,16034.483140,16452.290110,16763.991060,17065.027870,17334.755050,17524.223330,17554.090370,17477.643110,17326.918050,17180.826080,16996.586010,16671.965810,16334.979840,16073.692800,15898.027230,15846.324030,15777.126770,15728.800740,15704.780040,15693.239300
1,E92000001,England,16-19,One person households: Female,22469.415990,22440.586930,22653.577300,22867.484870,22858.524350,22502.335780,22442.590570,22219.994410,21918.766890,21460.190380,20299.416410,19516.264910,19012.579530,18512.062340,18154.698620,17673.516820,17041.041870,16471.727740,15988.333420,15800.004640,15762.620680,16301.864040,16846.653930,17420.782570,17886.222970,18239.519380,18597.022170,18909.107280,19099.550910,19119.890430,19022.983140,18839.212230,18688.783370,18481.094930,18145.068160,17788.909720,17512.495040,17335.226430,17275.491550,17196.055660,17139.634410,17110.248870,17094.658820
2,E92000001,England,16-19,Households with one dependent child,24438.364330,24612.193070,24971.972270,25315.211610,25351.737280,25190.598320,25375.984970,25328.096950,25180.931460,24843.888770,23715.748340,22972.667450,22532.432730,22064.419010,21741.548290,21251.636480,20559.726570,19935.976000,19371.270950,19179.527640,19177.705990,19819.898760,20461.025710,21136.882380,21667.766230,22064.910610,22463.430050,22792.693330,22974.976290,22948.626450,22777.413910,22506.800250,22269.231920,21980.069260,21552.978170,21107.266150,20768.530690,20564.993670,20522.975940,20462.487890,20428.944070,20425.226870,20439.401700
3,E92000001,England,16-19,Households with two dependent children,3756.336149,3765.193249,3802.747246,3837.299982,3825.947566,3801.614831,3827.394309,3813.971082,3787.672142,3736.107155,3567.469592,3456.156216,3388.786679,3317.074034,3268.147400,3193.996014,3090.479676,2997.832930,2914.875974,2888.106135,2891.219176,2988.701682,3086.418875,3189.364221,3270.365687,3331.174654,3392.179137,3442.393530,3470.377358,3467.170460,3442.144297,3401.968607,3366.248196,3322.624935,3257.957664,3190.290486,3138.847693,3108.017951,3101.803090,3092.786476,3087.721737,3087.211950,3089.392277
4,E92000001,England,16-19,Households with three or more dependent children,1160.096879,1120.597166,1092.320386,1063.486012,1023.250942,991.193379,972.368993,942.620142,912.609919,880.006852,823.308903,782.134630,751.889136,722.493790,699.692686,672.910565,641.263470,613.698308,589.284020,577.231192,571.912060,590.986482,610.144986,630.477900,646.453068,658.345964,670.259688,680.067864,685.494755,684.799378,679.694730,671.453154,664.082508,655.240120,642.297042,628.862925,618.603471,612.491781,611.420447,609.829765,609.006652,609.036026,609.585483


In [37]:
#only need row data where 'all ages' in age group column
num_hh_type = num_hh_type.drop(num_hh_type[num_hh_type['AGE GROUP'] != 'All ages'].index)

# Year goes up to 2043. Don't need past 2021
num_hh_type = num_hh_type.drop(num_hh_type.iloc[: , 25:48], axis = 1)

# rearrange layout so that years in rows and household types are columns.
# no longer need the age group column
num_hh_type = pd.melt(num_hh_type, id_vars= ['CODE', 'AREA NAME', 'AGE GROUP', 'HOUSEHOLD TYPE'],
       var_name = "year",value_name = 'num_hh' )

num_hh_type = num_hh_type.pivot(columns = 'HOUSEHOLD TYPE', index = ['CODE',
                                                                    'AREA NAME',
                                                                    'AGE GROUP',
                                                                    'year'], values = 'num_hh').fillna(0).reset_index()  

#delete age group & rename others
num_hh_type = num_hh_type.drop(columns = ['AGE GROUP']).rename(columns = {'CODE': 'la_code',
                                                                         'AREA NAME' : 'la_name',
                                                                         'Households with one dependent child': 'num_hh_1dp',
                                                                         'Households with three or more dependent children': 'num_hh_>2dp',
                                                                         'Households with two dependent children': 'num_hh_2dp',
                                                                         'One person households: Female': 'num_hh_1f',
                                                                         'One person households: Male': 'num_hh_1m',
                                                                         'Other households with two or more adults': 'num_hh_oth',
                                                                         'Total': 'num_hh_tot'})
pop_la['la_code'] = pop_la['la_code'].str.strip()
pop_la['la_name'] = pop_la['la_name'].str.strip()
num_hh_type['year'] = num_hh_type['year'].astype('int64')
#num_hh_type.head()

In [38]:
num_hh_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7056 entries, 0 to 7055
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   la_code      7056 non-null   object 
 1   la_name      7056 non-null   object 
 2   year         7056 non-null   int64  
 3   num_hh_1dp   7056 non-null   float64
 4   num_hh_>2dp  7056 non-null   float64
 5   num_hh_2dp   7056 non-null   float64
 6   num_hh_1f    7056 non-null   float64
 7   num_hh_1m    7056 non-null   float64
 8   num_hh_oth   7056 non-null   float64
 9   num_hh_tot   7056 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 551.4+ KB


In [39]:
la_file = la_file.merge(num_hh_type)
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4606 entries, 0 to 4605
Data columns (total 47 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   la_code               4606 non-null   object 
 1   la_name               4606 non-null   object 
 2   year                  4606 non-null   int64  
 3   g_belt_area           4606 non-null   int64  
 4   total_area            4606 non-null   int64  
 5   rpi                   4606 non-null   float64
 6   int_rate_max          4606 non-null   float64
 7   int_rate_min          4606 non-null   float64
 8   median_income         4606 non-null   int64  
 9   all_new_sls_price     4606 non-null   int64  
 10  det_new_sls_price     4606 non-null   int64  
 11  semi_new_sls_price    4606 non-null   int64  
 12  terr_new_sls_price    4606 non-null   int64  
 13  flat_new_sls_price    4606 non-null   int64  
 14  all_exist_sls_price   4606 non-null   int64  
 15  det_exist_sls_price  

### Adding: GDP

In [40]:
gdp_la = pd.read_csv('gdp by la.csv', header = 1)
gdp_la.head()

,ITL1 Region,LA code,LA name,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,North East,E06000001,Hartlepool,915,968,988,999,"1,054","1,114","1,159","1,205","1,262","1,326","1,351","1,325","1,357","1,411","1,416","1,461","1,448","1,543","1,654","1,675","1,674","1,775","1,757",1733
1,North East,E06000004,Stockton-on-Tees,"2,875","3,013","3,104","3,153","3,323","3,567","3,825","3,948","4,267","4,488","4,541","4,693","4,699","4,745","4,741","4,918","5,085","5,363","5,482","5,364","5,522","5,898","5,610",5759
2,North East,E06000002,Middlesbrough,"1,700","1,771","1,822","1,843","1,912","2,031","2,184","2,332","2,472","2,545","2,613","2,645","2,621","2,702","2,777","2,770","2,860","2,999","3,143","3,231","3,449","3,464","3,295",3382
3,North East,E06000003,Redcar and Cleveland,"1,463","1,491","1,531","1,516","1,560","1,721","1,752","1,799","1,830","1,850","1,954","1,927","2,010","2,048","2,072","2,073","2,146","2,223","2,174","2,043","2,125","2,233","2,182",2180
4,North East,E06000005,Darlington,"1,693","1,739","1,873","1,982","2,112","2,251","2,402","2,393","2,462","2,423","2,542","2,419","2,493","2,593","2,566","2,595","2,912","3,048","2,924","2,911","2,932","2,975","2,870",2905


In [41]:
gdp_la = gdp_la.drop(columns = 'ITL1 Region')
gdp_la = gdp_la.rename(columns = {'LA code': 'la_code','LA name': 'la_name'})
gdp_la = pd.melt(gdp_la, id_vars = ['la_code', 'la_name'], var_name = 'year', value_name = 'gdp')
gdp_la['gdp']  = gdp_la['gdp'].replace({',': ''}, regex=True).astype('int64')
gdp_la['year'] = gdp_la['year'].astype('int64')
gdp_la['la_code'] = gdp_la['la_code'].str.strip()
gdp_la['la_name'] = gdp_la['la_name'].str.strip()
#gdp_la.head()          

In [42]:
#gdp_la.info()
gdp_la.tail()

,la_code,la_name,year,gdp
8971,S12000026,Scottish Borders,2021,2765
8972,S12000006,Dumfries and Galloway,2021,4142
8973,S12000008,East Ayrshire,2021,2267
8974,S12000028,South Ayrshire,2021,2837
8975,S12000029,South Lanarkshire,2021,7840


In [43]:
la_file = la_file.merge(gdp_la)
#la_file.info()

### Adding: Dwelling Stock

In [44]:
la_stock = pd.read_csv('Live_Table_116 - LA dwelling stock 94-21.csv', header =3)
la_stock.head()

,Unnamed: 0,Current\nONS code,Lower and Single Tier Authority Data,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,England,E92000001,NaN,"3,666,190","3,564,590","3,469,594","3,400,635","3,309,244","3,177,507","3,011,995","2,811,931","2,706,036","2,456,917","2,334,631","2,165,526","2,087,456","1,987,343","1,870,366","1,819,696","1,786,427","1,725,912","1,692,625","1,681,782","1,668,683","1,643,256","1,612,325","1,601,550","1,592,151","1,587,164","1,582,946","1,580,811"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,North East,E12000001,NaN,"313,662","307,647","302,331","297,541","292,031","286,355","276,290","233,241","226,593","207,311","189,679","169,761","165,141","154,263","139,643","129,316","128,157","116,196","115,983","112,833","112,004","112,054","92,623","91,790","91,141","90,053","89,524","89,223"
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
la_stock = la_stock.drop(la_stock.index[0:5])
la_stock = la_stock.drop(columns =['Unnamed: 0'])
la_stock = la_stock.replace('..', np.nan)
la_stock = la_stock.rename(columns = {'Current\nONS code': 'la_code',
                                     'Lower and Single Tier Authority Data': 'la_name'})
la_stock = pd.melt(la_stock, id_vars= ['la_code', 'la_name'], var_name = "year",value_name = 'dwell_stock')
la_stock = la_stock.replace("..", 0)
la_stock.dropna(inplace =True)
la_stock['dwell_stock']  = la_stock['dwell_stock'].replace({',': ''}, regex=True).astype('float')
la_stock['year'] = la_stock['year'].astype('int64')
la_stock['la_code'] =la_stock['la_code'].str.strip()
la_stock['la_name'] =la_stock['la_name'].str.strip()

In [46]:
la_stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9555 entries, 0 to 11598
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   la_code      9555 non-null   object 
 1   la_name      9555 non-null   object 
 2   year         9555 non-null   int64  
 3   dwell_stock  9555 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 373.2+ KB


In [47]:
la_file = la_file.merge(la_stock)
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4520 entries, 0 to 4519
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   la_code               4520 non-null   object 
 1   la_name               4520 non-null   object 
 2   year                  4520 non-null   int64  
 3   g_belt_area           4520 non-null   int64  
 4   total_area            4520 non-null   int64  
 5   rpi                   4520 non-null   float64
 6   int_rate_max          4520 non-null   float64
 7   int_rate_min          4520 non-null   float64
 8   median_income         4520 non-null   int64  
 9   all_new_sls_price     4520 non-null   int64  
 10  det_new_sls_price     4520 non-null   int64  
 11  semi_new_sls_price    4520 non-null   int64  
 12  terr_new_sls_price    4520 non-null   int64  
 13  flat_new_sls_price    4520 non-null   int64  
 14  all_exist_sls_price   4520 non-null   int64  
 15  det_exist_sls_price  

### Adding: Net Additions

In [48]:
net_addt = pd.read_csv('Live_Table_122 - net additions by LA.csv', header = 3)
net_addt.head()

,Unnamed: 0,DCLG code,Former\nONS code,Current\nONS code,Met and Shire County Totals,Lower and Single Tier Authority Data,Unnamed: 6,2001-02,2002-03,2003-04,2004-05,2005-06,2006-07,2007-08,2008-09,2009-10,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20R,2020-21
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,England,NaN,NaN,E92000001,NaN,NaN,NaN,"146,704","159,875","170,969","185,553","202,653","214,936","223,534","182,767","144,870","137,394","134,896","124,722","136,605","170,693","189,645","217,345","222,281","241,877","242,702","216,489"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Unitary Authorities,NaN,NaN,NaN,NaN,NaN,NaN,"25,553","27,553","28,081","31,389","34,403","37,389","37,529","30,152","33,758","34,154","32,431","31,151","32,404","41,791","43,971","49,340","54,315","54,899","55,048","50,655"
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,20,21,22,23


In [49]:
net_addt = net_addt.drop(net_addt.index[0:5])
net_addt = net_addt.drop(columns = ['Unnamed: 0','DCLG code', 'Former\nONS code', 'Met and Shire County Totals', 'Unnamed: 6'])


#create list of years for dict to change to singular year
net_addt_col_lst = net_addt.columns.values.tolist()
net_addt_col_lst = net_addt_col_lst[2:]
net_addt_col_lst

d = {}
yr = 2002
for i in net_addt_col_lst:
    d[i] = yr
    yr+=1    
    
net_addt = net_addt.rename(columns = d)   

# This file has different descriptions for some la_names - suffixed with _ua. 
# Therefore named differently so not used in merge, only la_code used for the merge
# need to remove after merge
net_addt = net_addt.rename(columns = {'Current\nONS code': 'la_code',
                                     'Lower and Single Tier Authority Data': 'la_name_ua'})
net_addt = pd.melt(net_addt, id_vars= ['la_code', 'la_name_ua'], var_name = "year",value_name = 'net_additions').fillna(0)

# there are two rows that contain some kind of note. remove these
# must be a better way...cannot seem to convert to int without removing them. astype doesn't have coerce as 
#argument to be able to convert errors to Nan.

net_addt = net_addt.drop(net_addt[net_addt['net_additions'] == 'Latest update'].index)
net_addt = net_addt.drop(net_addt[net_addt['net_additions'] == 'Next update'].index)
net_addt = net_addt.drop(net_addt[net_addt['net_additions'] == '25-Nov-2021'].index)
net_addt = net_addt.drop(net_addt[net_addt['net_additions'] == 'Nov 2022'].index)
net_addt = net_addt.drop(net_addt[net_addt['net_additions'] == '..'].index)   

net_addt['net_additions'] = net_addt['net_additions'].replace({',': ''}, regex=True).astype('int64')

net_addt['year'] = net_addt['year'].astype('int64')
net_addt['la_code'] =net_addt['la_code'].str.strip()




In [50]:
net_addt['year'].unique()

array([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [51]:
#net_addt.tail()

In [52]:
la_file = la_file.merge(net_addt)
la_file = la_file.drop(columns = 'la_name_ua')
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4314 entries, 0 to 4313
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   la_code               4314 non-null   object 
 1   la_name               4314 non-null   object 
 2   year                  4314 non-null   int64  
 3   g_belt_area           4314 non-null   int64  
 4   total_area            4314 non-null   int64  
 5   rpi                   4314 non-null   float64
 6   int_rate_max          4314 non-null   float64
 7   int_rate_min          4314 non-null   float64
 8   median_income         4314 non-null   int64  
 9   all_new_sls_price     4314 non-null   int64  
 10  det_new_sls_price     4314 non-null   int64  
 11  semi_new_sls_price    4314 non-null   int64  
 12  terr_new_sls_price    4314 non-null   int64  
 13  flat_new_sls_price    4314 non-null   int64  
 14  all_exist_sls_price   4314 non-null   int64  
 15  det_exist_sls_price  

In [53]:
la_file.tail()

,la_code,la_name,year,g_belt_area,total_area,rpi,int_rate_max,int_rate_min,median_income,all_new_sls_price,det_new_sls_price,semi_new_sls_price,terr_new_sls_price,flat_new_sls_price,all_exist_sls_price,det_exist_sls_price,semi_exist_sls_price,terr_exist_sls_price,flat_exist_sls_price,all_sls_price,det_sls_price,semi_sls_price,terr_sls_price,flat_sls_price,all_new_sls_num,det_new_sls_num,semi_new_sls_num,terr_new_sls_num,flat_new_sls_num,all_exist_sls_num,det_exist_sls_num,semi_exist_sls_num,terr_exist_sls_num,flat_exist_sls_num,all_sls_num,det_sls_num,semi_sls_num,terr_sls_num,flat_sls_num,population,num_hh_1dp,num_hh_>2dp,num_hh_2dp,num_hh_1f,num_hh_1m,num_hh_oth,num_hh_tot,gdp,dwell_stock,net_additions
4309,E06000054,Wiltshire,2021,6910,325530,305.0,0.25,0.25,29069,319995,387000,272000,234995,284500,300000,425000,274000,235000,152000,300000,420000,273000,235000,155000,784,387,246,111,40,8992,3454,2323,2440,775,9776,3841,2569,2551,815,510400,27529.944330,9144.090624,24156.774210,33613.031770,26084.193350,95427.74348,215955.77780,13123,5335.0,2846
4310,E07000094,Winchester,2021,0,66100,305.0,0.25,0.25,33601,382500,472473,349000,341473,745000,475000,675000,412500,378265,234250,470500,665000,400000,370000,237250,155,58,41,40,16,2308,923,533,582,270,2463,981,574,622,286,127500,5926.207018,2201.551867,5788.467309,8163.926710,6045.905833,22725.37194,50851.43068,5953,5117.0,804
4311,E06000041,Wokingham,2021,2900,17900,305.0,0.25,0.25,38014,440000,565530,430000,425000,279000,450000,595000,425000,350000,230000,445975,590000,425000,355000,236250,319,124,117,46,32,3129,1273,881,663,312,3448,1397,998,709,344,177500,9435.057511,2765.600972,9610.106201,9370.879657,7060.580500,29136.29719,67378.52203,8544,2663.0,1129
4312,E07000238,Wychavon,2021,8860,66350,305.0,0.25,0.25,28865,342000,364750,255000,252500,369500,297250,407500,252500,217200,130000,302500,400000,252500,219750,137000,207,116,54,22,15,2270,1075,609,458,128,2477,1191,663,480,143,132500,6720.508791,1695.917715,5903.666073,8182.339119,6899.943500,27087.69718,56490.07238,3474,6.0,591
4313,E06000014,York,2021,22410,27190,305.0,0.25,0.25,32490,350000,420000,332745,285000,324995,272000,395000,266000,245000,175000,275000,395000,268200,246000,180000,119,28,18,8,65,3681,831,1209,1167,474,3800,859,1227,1175,539,202800,11058.879810,2525.976051,7466.497741,14185.253680,12707.300200,39631.93014,87575.83763,6954,7527.0,622


### Adding: Number of Households on Housing Waiting List

In [54]:
on_wait_lst = pd.read_csv('LT600 - No households on housing waiting list by LA.csv', header = 3)
#on_wait_lst.head(10)

In [55]:
on_wait_lst = on_wait_lst.drop(on_wait_lst.index[0:5])
on_wait_lst = on_wait_lst.drop(columns =['Unnamed: 0'])
on_wait_lst = on_wait_lst.replace('..', np.nan)
on_wait_lst = on_wait_lst.replace('.', np.nan)

#There is inconsistency with la_names versus other files. And some additional codes, a few missing
#Don't use la_name for merge. label differently and remove
on_wait_lst = on_wait_lst.rename(columns = {'Current\nONS code': 'la_code',
                                     'Lower and Single Tier Authority Data': 'la_name_'})
on_wait_lst = pd.melt(on_wait_lst, id_vars= ['la_code', 'la_name_'], var_name = "year",value_name = 'num_wait_lst')
on_wait_lst['num_wait_lst']  = on_wait_lst['num_wait_lst'].replace({',': ''}, regex=True).astype('float')
on_wait_lst['year'] = on_wait_lst['year'].astype('int64')
on_wait_lst['la_code'] = on_wait_lst['la_code'].str.strip()


on_wait_lst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14735 entries, 0 to 14734
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   la_code       13230 non-null  object 
 1   la_name_      12950 non-null  object 
 2   year          14735 non-null  int64  
 3   num_wait_lst  12301 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 460.6+ KB


In [56]:
la_file = la_file.merge(on_wait_lst)
la_file = la_file.drop(columns = 'la_name_')
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4314 entries, 0 to 4313
Data columns (total 51 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   la_code               4314 non-null   object 
 1   la_name               4314 non-null   object 
 2   year                  4314 non-null   int64  
 3   g_belt_area           4314 non-null   int64  
 4   total_area            4314 non-null   int64  
 5   rpi                   4314 non-null   float64
 6   int_rate_max          4314 non-null   float64
 7   int_rate_min          4314 non-null   float64
 8   median_income         4314 non-null   int64  
 9   all_new_sls_price     4314 non-null   int64  
 10  det_new_sls_price     4314 non-null   int64  
 11  semi_new_sls_price    4314 non-null   int64  
 12  terr_new_sls_price    4314 non-null   int64  
 13  flat_new_sls_price    4314 non-null   int64  
 14  all_exist_sls_price   4314 non-null   int64  
 15  det_exist_sls_price  

### Adding: Affordable Supply

In [57]:
afford_supply = pd.read_csv('live table 1011 data - affordable housing supply 91-21.csv')
afford_supply.head()

,LA code,LA name,District code,District name,Metropolitan code,Metropolitan name,Region code,Region name,LA code 202021,LA name 202021,LA type 202021,Year,Tenure,Completions,s106,Type,Provider,LT1000,Units
0,E06000001,Hartlepool,NaN,NaN,NaN,NaN,E12000001,North East,E06000001,Hartlepool,E06,1991-92,Affordable Home Ownership,Completion,N,AQ,RP,Private Registered Provider HE/GLA funded,13
1,E06000001,Hartlepool,NaN,NaN,NaN,NaN,E12000001,North East,E06000001,Hartlepool,E06,1991-92,Social Rent,Completion,N,NB,LA,Local Authority HE/GLA funded,72
2,E06000001,Hartlepool,NaN,NaN,NaN,NaN,E12000001,North East,E06000001,Hartlepool,E06,1991-92,Social Rent,Completion,N,NB,RP,Private Registered Provider HE/GLA funded,57
3,E06000001,Hartlepool,NaN,NaN,NaN,NaN,E12000001,North East,E06000001,Hartlepool,E06,1991-92,Social Rent,Completion,N,AQ,RP,Private Registered Provider HE/GLA funded,5
4,E06000002,Middlesbrough,NaN,NaN,NaN,NaN,E12000001,North East,E06000002,Middlesbrough,E06,1991-92,Affordable Home Ownership,Completion,N,NB,RP,Private Registered Provider HE/GLA funded,18


In [58]:
# a number of features are not required. Remove the columns
afford_col_list = afford_supply.columns.values.tolist()
afford_col_list

['LA code',
 'LA name',
 'District code',
 'District name',
 'Metropolitan code',
 'Metropolitan name',
 'Region code',
 'Region name',
 'LA code 202021',
 'LA name 202021',
 'LA type 202021',
 'Year',
 'Tenure',
 'Completions',
 's106',
 'Type',
 'Provider',
 'LT1000',
 'Units']

In [59]:
afford_supply = afford_supply.drop(columns = [ 'District code',
                                              'District name',
                                              'Metropolitan code',
                                              'Metropolitan name', 
                                              'Region code',
                                              'Region name',
                                              'LA code 202021',
                                              'LA name 202021',
                                              'LA type 202021', 
                                              'Type',
                                              'Provider',
                                              'LT1000',
                                              's106'])

# only require completions, as starts not actually contributing to housing needs.
afford_supply = afford_supply.drop(afford_supply[afford_supply['Completions'] == 'Start'].index)

# change year to singular number
def yr_func(x):
    return int(x[:4])+1
    
afford_supply['Year'] = afford_supply['Year'].apply(yr_func)

afford_supply = afford_supply.groupby(['LA code', 
                                       'LA name',
                                       'Year',
                                       'Tenure'])['Units'].sum().reset_index()

#for tidy data, need tenure to be a feature/column and no of units to be the measure for each type of tenure
afford_supply = afford_supply.pivot(columns = 'Tenure', index = ['LA code', 
                                                                 'LA name',
                                                                 'Year'], values = 'Units').fillna(0).reset_index() 
afford_supply = afford_supply.rename(columns = {'Affordable Home Ownership': 'aff_own',
                                               'Affordable Rent': 'aff_rent',
                                               'Intermediate Rent': 'aff_int_rent',
                                               'London Affordable Rent': 'aff_lon_rent',
                                               'Shared Ownership': 'aff_sh_own',
                                               'Social Rent': 'aff_soc_rent',
                                               'Unknown': 'aff_unkwn',
                                               'LA code': 'la_code',
                                               'LA name':'la_name_',
                                               'Year': 'year'})

on_wait_lst['la_code'] = on_wait_lst['la_code'].str.strip()

In [60]:
afford_supply.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10205 entries, 0 to 10204
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   la_code       10205 non-null  object 
 1   la_name_      10205 non-null  object 
 2   year          10205 non-null  int64  
 3   aff_own       10205 non-null  float64
 4   aff_rent      10205 non-null  float64
 5   aff_int_rent  10205 non-null  float64
 6   aff_lon_rent  10205 non-null  float64
 7   aff_sh_own    10205 non-null  float64
 8   aff_soc_rent  10205 non-null  float64
 9   aff_unkwn     10205 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 797.4+ KB


In [61]:
la_file = la_file.merge(afford_supply)
la_file = la_file.drop(columns = 'la_name_')
la_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4291 entries, 0 to 4290
Data columns (total 58 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   la_code               4291 non-null   object 
 1   la_name               4291 non-null   object 
 2   year                  4291 non-null   int64  
 3   g_belt_area           4291 non-null   int64  
 4   total_area            4291 non-null   int64  
 5   rpi                   4291 non-null   float64
 6   int_rate_max          4291 non-null   float64
 7   int_rate_min          4291 non-null   float64
 8   median_income         4291 non-null   int64  
 9   all_new_sls_price     4291 non-null   int64  
 10  det_new_sls_price     4291 non-null   int64  
 11  semi_new_sls_price    4291 non-null   int64  
 12  terr_new_sls_price    4291 non-null   int64  
 13  flat_new_sls_price    4291 non-null   int64  
 14  all_exist_sls_price   4291 non-null   int64  
 15  det_exist_sls_price  

In [62]:
la_file.tail()

,la_code,la_name,year,g_belt_area,total_area,rpi,int_rate_max,int_rate_min,median_income,all_new_sls_price,det_new_sls_price,semi_new_sls_price,terr_new_sls_price,flat_new_sls_price,all_exist_sls_price,det_exist_sls_price,semi_exist_sls_price,terr_exist_sls_price,flat_exist_sls_price,all_sls_price,det_sls_price,semi_sls_price,terr_sls_price,flat_sls_price,all_new_sls_num,det_new_sls_num,semi_new_sls_num,terr_new_sls_num,flat_new_sls_num,all_exist_sls_num,det_exist_sls_num,semi_exist_sls_num,terr_exist_sls_num,flat_exist_sls_num,all_sls_num,det_sls_num,semi_sls_num,terr_sls_num,flat_sls_num,population,num_hh_1dp,num_hh_>2dp,num_hh_2dp,num_hh_1f,num_hh_1m,num_hh_oth,num_hh_tot,gdp,dwell_stock,net_additions,num_wait_lst,aff_own,aff_rent,aff_int_rent,aff_lon_rent,aff_sh_own,aff_soc_rent,aff_unkwn
4286,E06000054,Wiltshire,2021,6910,325530,305.0,0.25,0.25,29069,319995,387000,272000,234995,284500,300000,425000,274000,235000,152000,300000,420000,273000,235000,155000,784,387,246,111,40,8992,3454,2323,2440,775,9776,3841,2569,2551,815,510400,27529.944330,9144.090624,24156.774210,33613.031770,26084.193350,95427.74348,215955.77780,13123,5335.0,2846,3947.0,10.0,351.0,0.0,0.0,297.0,61.0,0.0
4287,E07000094,Winchester,2021,0,66100,305.0,0.25,0.25,33601,382500,472473,349000,341473,745000,475000,675000,412500,378265,234250,470500,665000,400000,370000,237250,155,58,41,40,16,2308,923,533,582,270,2463,981,574,622,286,127500,5926.207018,2201.551867,5788.467309,8163.926710,6045.905833,22725.37194,50851.43068,5953,5117.0,804,1452.0,0.0,30.0,0.0,0.0,62.0,26.0,0.0
4288,E06000041,Wokingham,2021,2900,17900,305.0,0.25,0.25,38014,440000,565530,430000,425000,279000,450000,595000,425000,350000,230000,445975,590000,425000,355000,236250,319,124,117,46,32,3129,1273,881,663,312,3448,1397,998,709,344,177500,9435.057511,2765.600972,9610.106201,9370.879657,7060.580500,29136.29719,67378.52203,8544,2663.0,1129,2086.0,0.0,17.0,0.0,0.0,193.0,92.0,0.0
4289,E07000238,Wychavon,2021,8860,66350,305.0,0.25,0.25,28865,342000,364750,255000,252500,369500,297250,407500,252500,217200,130000,302500,400000,252500,219750,137000,207,116,54,22,15,2270,1075,609,458,128,2477,1191,663,480,143,132500,6720.508791,1695.917715,5903.666073,8182.339119,6899.943500,27087.69718,56490.07238,3474,6.0,591,1242.0,6.0,95.0,0.0,0.0,83.0,147.0,0.0
4290,E06000014,York,2021,22410,27190,305.0,0.25,0.25,32490,350000,420000,332745,285000,324995,272000,395000,266000,245000,175000,275000,395000,268200,246000,180000,119,28,18,8,65,3681,831,1209,1167,474,3800,859,1227,1175,539,202800,11058.879810,2525.976051,7466.497741,14185.253680,12707.300200,39631.93014,87575.83763,6954,7527.0,622,1738.0,6.0,57.0,0.0,0.0,28.0,73.0,0.0


In [64]:
la_file.to_csv('la_file.csv')

In [65]:
la_file['year'].unique()

array([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)